### Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Data Load

In [2]:
import glob
import os

img_lst = glob.glob("/content/drive/MyDrive/Colab_Notebooks/안벽/Prototype_v3/train/images/"+"*.jpg")+glob.glob("/content/drive/MyDrive/Colab_Notebooks/안벽/Prototype_v3/train/images/"+"*.JPG")+glob.glob("/content/drive/MyDrive/Colab_Notebooks/안벽/Prototype_v3/train/images/"+"*.jpeg")
bbox_lst = glob.glob("/content/drive/MyDrive/Colab_Notebooks/안벽/Prototype_v3/train/labels/"+"*.txt")

img_lst.sort()
bbox_lst.sort()

len(img_lst), len(bbox_lst)

(791, 791)

### Function Define
v4,v5와 비교를 위해 791 모두를 증강하는 버전과 <br>
이후를 위해 새로 만들 이미지 갯수를 지정하는 버전을 나눴습니다.<br>
내용의 수정을 거치면 여러가지 증강 방법을 랜덤으로 선택하도록 할 수 있을듯합니다.

In [ ]:
from tqdm import tqdm

sample_preds_folder = "/content/drive/MyDrive/Colab_Notebooks/안벽/Prototype_v6/"
dir_img = "noise_img/"
dir_bbox = "noise_bbox/"

In [4]:
import cv2
import copy
from random import randint
import random
import numpy as np
 
def SaltPepper(img):
    # Getting the dimensions of the image
    if img.ndim > 2:  # color
        height, width, _ = img.shape
    else:  # gray scale
        height, width = img.shape
 
    result = copy.deepcopy(img)
 
    # Randomly pick some pixels in the image
    # Pick a random number between height*width/80 and height*width/10
    number_of_pixels = randint(int(height * width / 100), int(height * width / 10))
 
    for i in range(number_of_pixels):
        # Pick a random y coordinate
        y_coord = randint(0, height - 1)
 
        # Pick a random x coordinate
        x_coord = randint(0, width - 1)
 
        if result.ndim > 2:
            result[y_coord][x_coord] = [randint(0, 255), randint(0, 255), randint(0, 255)]
        else:
            # Color that pixel to white
            result[y_coord][x_coord] = 255
 
    # Randomly pick some pixels in image
    # Pick a random number between height*width/80 and height*width/10
    for i in range(number_of_pixels):
        # Pick a random y coordinate
        y_coord = randint(0, height - 1)
 
        # Pick a random x coordinate
        x_coord = randint(0, width - 1)
 
        if result.ndim > 2:
            result[y_coord][x_coord] = [randint(0, 255), randint(0, 255), randint(0, 255)]
        else:
            # Color that pixel to white
            result[y_coord][x_coord] = 0
 
    return result

In [ ]:
random.seed(1031)

def image_addNoise(img_item):
  #노이즈 랜덤 지정
  random_augment = random.randrange(1,3)
  #이미지 로드
  im = cv2.imread(img_item)
  row,col,ch= im.shape

  if(random_augment == 1):
    #솔트페퍼 노이즈 추가하기
    result = SaltPepper(im)
    transformed_image = result      
      
  elif(random_augment == 2):
    #가우시안 노이즈 추가하기
    mean = 0
    var = 0.1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy_array = im + gauss
    transformed_image = noisy_array
  
  return transformed_image

### Image Data Preprocessing & Save

In [5]:
import os

def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [ ]:
# Create Directory

createDirectory(sample_preds_folder+dir_img)
createDirectory(sample_preds_folder+dir_bbox)

In [ ]:
import shutil

# 전처리
for img_item, bbox_item in tqdm(zip(img_lst,bbox_lst)):

    transformed_image = image_addNoise(img_item) 
    cv2.imwrite(os.path.join(sample_preds_folder,dir_img,"{}{}.jpg".format(img_item[72:-4],"_n")),transformed_image) # 새로운 이미지 저장     

    f_name = os.path.join(sample_preds_folder,dir_bbox,"{}{}.txt".format(bbox_item[72:-4],"_n"))
    shutil.copy(bbox_item, f_name)

791it [24:34,  1.86s/it]


----------------------------------------------------------------

밑의 지정버전 문제점은 같은 인덱스가 선택되어 <br>
1) 같은 사진+다른 방법 <br>
2) 같은 사진+같은 방법 <br>
의 경우도 생긴다는 점 

In [26]:
import cv2
import numpy as np
import random
import shutil
random.seed(1031)

from tqdm import tqdm

sample_preds_folder = "/content/drive/MyDrive/Colab_Notebooks/안벽/Prototype_v6/"
dir_img = "랜덤실험용_img/"
dir_bbox = "랜덤실험용_bbox/"

# Create Directory

createDirectory(sample_preds_folder+dir_img)
createDirectory(sample_preds_folder+dir_bbox)


def random_aug(num_augmented_images,total_origin_image_num,augment_cnt):
  for i in range(0, num_augmented_images):
      change_picture_index = random.randrange(0, total_origin_image_num-1)
      file_name = img_lst[change_picture_index]
      txt_name = bbox_lst[change_picture_index]
      #노이즈 랜덤 지정
      random_augment = random.randrange(1,3)
      #이미지 로드
      im = cv2.imread(file_name)
      row,col,ch= im.shape

      #조건문 안의 내용을 여러가지 다른 방법으로 바꾸는 방향으로 수정 가능할듯~    
      if(random_augment == 1):
        #솔트페퍼 노이즈 추가하기
        result = SaltPepper(im)
        transformed_image = result   
        cv2.imwrite(os.path.join(sample_preds_folder,dir_img,"{}{}.jpg".format(file_name[72:-4],"_sp")),transformed_image)   
        label_name = os.path.join(sample_preds_folder,dir_bbox,"{}{}.txt".format(txt_name[72:-4],"_sp"))
        shutil.copy(txt_name, label_name)
          
      elif(random_augment == 2):
        #가우시안 노이즈 추가하기
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy_array = im + gauss
        transformed_image = noisy_array
        cv2.imwrite(os.path.join(sample_preds_folder,dir_img,"{}{}.jpg".format(file_name[72:-4],"_g")),transformed_image)
        label_name = os.path.join(sample_preds_folder,dir_bbox,"{}{}.txt".format(txt_name[72:-4],"_g"))
        shutil.copy(txt_name, label_name)
              
      augment_cnt += 1
      print(augment_cnt)

In [27]:
#다음 변수를 수정하여 새로 만들 이미지 갯수를 지정
num_augmented_images = 5

total_origin_image_num = len(img_lst[:10])
augment_cnt = 0


#함수 실행
random_aug(num_augmented_images,total_origin_image_num,augment_cnt)

1
2
3
4
5
